<a href="https://colab.research.google.com/github/EricSiq/NASA-Space-Apps-Challenge/blob/main/Notebooks/DataIngestionDoclingTest1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%pip install langchain_docling docling

This is an excellent first step in your project\! Yes, it is **absolutely possible** and is the correct workflow to prepare your data for **Docling**.

You must first extract the links and their titles from the CSV because **Docling** (or any document loader) requires the specific file path or URL for each document it processes.

## Best Code Approach: Extract and Prepare

The best approach is to use the `pandas` library to load the CSV, extract the necessary columns, and structure the data into a list of tuples or a list of dictionaries, which is a perfect input format for your subsequent processing loop with Docling.

### Python Code for Extraction

This code will load your CSV and generate a clean list of (Title, Link) pairs, filtering for the valid PMC links.

In [4]:
import pandas as pd
import csv

def extract_publication_data(file_name):
    """
    Loads the CSV, extracts the Title and Link columns, and prepares a
    list of (title, link) tuples for processing.
    """
    try:
        # Load the CSV file
        df = pd.read_csv(file_name, quoting=csv.QUOTE_MINIMAL)

        # Assuming the columns are named 'Title' and 'Link'
        # Filter for links that start with the expected base URL for PMC articles
        base_url_prefix = "https://www.ncbi.nlm.nih.gov/pmc/articles/"
        df_filtered = df[df['Link'].str.startswith(base_url_prefix, na=False)].copy()

        # Create a list of tuples (Title, Link)
        publication_list = list(zip(df_filtered['Title'], df_filtered['Link']))

        return publication_list

    except Exception as e:
        print(f"An error occurred during file processing: {e}")
        return []

# File name provided in the user context
file_name = "SB_publication_PMC.csv"

# Run the extraction
publication_data = extract_publication_data(file_name)

# Print a sample of the extracted data for verification
if publication_data:
    print(f"Total publications extracted: {len(publication_data)}")
    print("Sample of extracted (Title, Link) pairs:")
    for i in range(min(5, len(publication_data))):
        print(f"  {i+1}. Title: {publication_data[i][0][:60]}... | Link: {publication_data[i][1]}")
else:
    print("No publication data was extracted.")

Total publications extracted: 607
Sample of extracted (Title, Link) pairs:
  1. Title: Mice in Bion-M 1 space mission: training and selection... | Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4136787/
  2. Title: Microgravity induces pelvic bone loss through osteoclastic a... | Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3630201/
  3. Title: Stem Cell Health and Tissue Regeneration in Microgravity... | Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11988870/
  4. Title: Microgravity Reduces the Differentiation and Regenerative Po... | Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7998608/
  5. Title: Microgravity validation of a novel system for RNA isolation ... | Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5587110/


-----

## Next Step: Docling Integration Workflow

Once the above code provides the `publication_data` list, you will use it in the next step to perform the actual ingestion and structural parsing using $\text{Docling}$.

1.  **Iterate:** Loop through the `publication_data` list.
2.  **Docling Call:** Inside the loop, initialize the $\text{DoclingLoader}$ using the **URL** from the list. Docling is capable of loading directly from a URL.
3.  **Process:** Use the `load()` or `lazy_load()` method to fetch and process the document.
4.  **Extract & Store:** Immediately process the structured $\text{Docling}$ output to perform your **NER/RE** and populate the **Knowledge Graph** and **Vector Store**.

### Conceptual Docling Integration (Python)

This snippet illustrates how the extraction output feeds into $\text{Docling}$ for processing:

In [16]:
#Assuming you successfully ran the extraction code above and have the 'publication_data' list

# --- Conceptual Code for Docling Ingestion (Requires Docling and LangChain setup) ---

from langchain_docling import DoclingLoader
from docling.chunking import HybridChunker
from docling.api import ExportType # Corrected import path

processed_documents = []

for title, url in publication_data:
    try:
        # 1. Initialize DoclingLoader with the URL
        loader = DoclingLoader(
            file_path=url,
            chunker=HybridChunker(),
            export_type=ExportType.MARKDOWN
        )

        # 2. Load the structured document
        docs = loader.load()

        # 3. Add original metadata (Title) to each chunk and store
        for doc in docs:
            doc.metadata['original_title'] = title
            processed_documents.append(doc)

        # 4. NOW: Run NER/RE and KG population on the 'docs' list

    except Exception as e:
        print(f"Failed to process document: {title} at {url}. Error: {e}")

# # The 'processed_documents' list is now ready for your Knowledge Graph pipeline.

ModuleNotFoundError: No module named 'docling.api'

After running the installation cell above, please execute the cell below again.

The approach is to **Extract $\rightarrow$ Transform $\rightarrow$ Load** (ETL), where $\text{Docling}$ handles the critical **Transform** step by converting the messy PDF structure into clean, semantically-rich text (like Markdown) before the downstream AI work begins.

In [20]:
import pandas as pd
import csv
import os
import time
from typing import List, Tuple, Dict, Any

# Docling Imports
# Import necessary classes from Docling and its LangChain integration

# Docling's specific classes for chunking and export types
from docling.chunking import HybridChunker
from langchain_docling.loader import DoclingLoader, ExportType
from langchain_core.documents import Document

# A. Function to Extract Data from CSV
def extract_publication_data(file_name: str) -> List[Tuple[str, str]]:
    """
    Loads the CSV, extracts the Title and Link columns, and filters for valid PMC URLs.
    """
    try:
        # Load the CSV file
        df = pd.read_csv(file_name, quoting=csv.QUOTE_MINIMAL)

        # Assuming the columns are named 'Title' and 'Link'
        base_url_prefix = "https://www.ncbi.nlm.nih.gov/pmc/articles/"

        # 1. Filter: Ensure we only keep valid links for PMC articles
        df_filtered = df[df['Link'].astype(str).str.startswith(base_url_prefix, na=False)].copy()

        # 2. Select: Create a list of tuples (Title, Link)
        publication_list = list(zip(df_filtered['Title'], df_filtered['Link']))

        print(f"✅ Extracted {len(publication_list)} valid PMC links from the CSV.")
        return publication_list

    except Exception as e:
        print(f"❌ An error occurred during CSV extraction: {e}")
        return []

# B. Function to Ingest Data using Docling
def ingest_documents_with_docling(publication_data: List[Tuple[str, str]]) -> List[Document]:
    """
    Iterates through the list of (title, url) and uses Docling to fetch and process
    each document, preserving structural information in Markdown format.
    """
    if not publication_data:
        print("No data to process. Exiting ingestion.")
        return []

    processed_documents: List[Document] = []

    # Initialize the chunking strategy once
    chunker = HybridChunker()

    # We will only process a small sample for demonstration/testing due to processing time.
    # For full production run, remove the slicing [0:5]
    sample_data = publication_data[0:5]
    print(f"\nProcessing a sample of {len(sample_data)} documents using Docling...")

    for i, (title, url) in enumerate(sample_data):
        start_time = time.time()
        print(f"--- Document {i+1}/{len(sample_data)}: {title[:50]}...")

        try:
            # 1. Initialize DoclingLoader with the URL
            # Docling handles fetching the content from the web
            loader = DoclingLoader(
                file_path=url,
                chunker=chunker,
                # Export as Markdown to preserve semantic structure (headings, tables)
                export_type=ExportType.MARKDOWN
            )

            # 2. Load the structured document. The result is a list of Document chunks.
            docs = loader.load()

            # 3. Add custom metadata (Title, Source URL) to each chunk
            for doc in docs:
                doc.metadata['original_title'] = title
                doc.metadata['source_url'] = url
                # Add a unique ID for traceability in the Knowledge Graph
                doc.metadata['doc_id'] = f"PMC_{url.split('/')[-2]}"
                processed_documents.append(doc)

            end_time = time.time()
            print(f"    ✅ Processed {len(docs)} chunks. Time: {end_time - start_time:.2f}s")
            # This is where you would call your NER/RE and KG population functions
            # process_for_knowledge_graph(docs)

        except Exception as e:
            print(f"    ❌ FAILED to process document. Error: {e}")

    return processed_documents

# Main Execution
if __name__ == "__main__":
    #  Phase 1: Extraction
    file_name = "SB_publication_PMC.csv"
    publication_data = extract_publication_data(file_name)

    #  Phase 2: Ingestion and Structural Parsing
    if publication_data:
        final_document_chunks = ingest_documents_with_docling(publication_data)

        # Verification of the processed chunks
        if final_document_chunks:
            print("\n--- Verification ---")
            print(f"Total structured chunks ready for KG/Vector Store: {len(final_document_chunks)}")

            # Show a sample chunk content and metadata
            sample_chunk = final_document_chunks[0]
            print(f"\nSample Chunk Content (First 200 chars): \n{sample_chunk.page_content[:200]}...")
            print("\nSample Chunk Metadata:")
            print(sample_chunk.metadata)

✅ Extracted 607 valid PMC links from the CSV.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]


Processing a sample of 5 documents using Docling...
--- Document 1/5: Mice in Bion-M 1 space mission: training and selec...


    ✅ Processed 1 chunks. Time: 2.49s
--- Document 2/5: Microgravity induces pelvic bone loss through oste...
    ✅ Processed 1 chunks. Time: 1.68s
--- Document 3/5: Stem Cell Health and Tissue Regeneration in Microg...
    ✅ Processed 1 chunks. Time: 1.59s
--- Document 4/5: Microgravity Reduces the Differentiation and Regen...


    ✅ Processed 1 chunks. Time: 1.23s
--- Document 5/5: Microgravity validation of a novel system for RNA ...
    ✅ Processed 1 chunks. Time: 1.51s

--- Verification ---
Total structured chunks ready for KG/Vector Store: 5

Sample Chunk Content (First 200 chars): 
# Mice in Bion-M 1 Space Mission: Training and Selection

[Alexander Andreev-Andrievskiy](https://pubmed.ncbi.nlm.nih.gov/?term=%22Andreev-Andrievskiy%20A%22%5BAuthor%5D)

1, 2, * , [Anfisa Popova](ht...

Sample Chunk Metadata:
{'source': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4136787/', 'original_title': 'Mice in Bion-M 1 space mission: training and selection', 'source_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4136787/', 'doc_id': 'PMC_PMC4136787'}
